In [42]:
import tkinter as tk
from tkinter import messagebox
import re

def number_to_thai(num):
    unit = ["", "หนึ่ง", "สอง", "สาม", "สี่", "ห้า", "หก", "เจ็ด", "แปด", "เก้า"]
    ten = ["", "สิบ", "ยี่สิบ", "สามสิบ", "สี่สิบ", "ห้าสิบ", "หกสิบ", "เจ็ดสิบ", "แปดสิบ", "เก้าสิบ"]
    
    def convert_integer_part(n):
        if n == 0:
            return "ศูนย์"
        
        parts = []
        if n >= 1000000:
            parts.append(unit[n // 1000000] + "ล้าน")
            n %= 1000000
        if n >= 100000:
            parts.append(unit[n // 100000] + "แสน")
            n %= 100000
        if n >= 10000:
            parts.append(unit[n // 10000] + "หมื่น")
            n %= 10000
        if n >= 1000:
            parts.append(unit[n // 1000] + "พัน")
            n %= 1000
        if n >= 100:
            parts.append(unit[n // 100] + "ร้อย")
            n %= 100
        if n == 10:
            parts.append("สิบ")
        if n > 10:
            parts.append(ten[n // 10])
            n %= 10
        if n > 0 and n <= 9:  # จัดการเลขหลักหน่วย
            if n == 1 and len(parts) > 0:
                parts.append("เอ็ด")
            else:
                parts.append(unit[n])
        return "".join(parts)

    def convert_decimal_part(n):
        if n > 0 and n <= 99:
            ten_part = n // 10
            unit_part = n % 10
            if unit_part == 0:
                return ten[ten_part] + "สตางค์"
            elif unit_part == 1 and ten_part > 0:
                return ten[ten_part] + "เอ็ด" + "สตางค์"
            else:
                return ten[ten_part] + unit[unit_part] + "สตางค์"
        return ""
    
    integer_part = int(num)
    decimal_part = round((num - integer_part) * 100) 

    # ถ้ามีเศษสตางค์มากกว่า 0
    result = convert_integer_part(integer_part) + "บาท"
    if decimal_part > 0:
        result += convert_decimal_part(decimal_part)
    else:
        result += "ถ้วน"
    return result

def validate_input(input_str):
    # ตรวจสอบรูปแบบตัวเลข เช่น 1234.25 หรือ 1,234.25
    pattern = r"^\d{1,3}(,\d{3})*(\.\d{1,2})?$"
    
    # ตรวจสอบว่าในส่วนของหลักสตางค์มีมากกว่า 2 หลักหรือไม่
    if '.' in input_str:
        decimal_part = input_str.split('.')[-1]
        if len(decimal_part) > 2:
            messagebox.showwarning("ข้อผิดพลาด", "ระบบรองรับหลักสตางค์ได้สูงสุด\nแค่ 2 หลัก หรือสูงสุด 99 สตางค์เท่านั้น")
            return False

    return re.match(pattern, input_str)

def convert():
    try:
        user_input = entry.get().strip()
        if not user_input:
            raise ValueError("กรุณากรอกจำนวนเงิน")
        
        # ตรวจสอบรูปแบบตัวเลข
        if not validate_input(user_input):
            return messagebox.showwarning("ข้อผิดพลาด", "รูปแบบตัวเลขไม่ถูกต้อง\n(ตัวอย่าง: 1234.25 หรือ 1,234.25)")
        
        # เอาเครื่องหมาย ',' ออกจากข้อความ
        sanitized_input = user_input.replace(",", "")
        number = float(sanitized_input)
        
        # ตรวจสอบขอบเขตของตัวเลข
        if number > 999999999.99:
            raise ValueError("ตัวเลขเกินขอบเขตที่โปรแกรมรองรับ (สูงสุด 999,999,999.99)")
        
        # แปลงตัวเลขเป็นคำภาษาไทย
        result = number_to_thai(number)
        result_label.config(text=f"ผลลัพธ์: {result}")
    except ValueError as e:
        messagebox.showerror("ข้อผิดพลาด", str(e))

# ส่วนของ GUI
root = tk.Tk()
root.title("แปลงตัวเลขเป็นคำภาษาไทย")

# ส่วนรับข้อมูล
tk.Label(root, text="กรอกจำนวนเงิน:").grid(row=0, column=0, padx=10, pady=10)
entry = tk.Entry(root, width=20)
entry.grid(row=0, column=1, padx=10, pady=10)

# ปุ่มแปลง
convert_button = tk.Button(root, text="แปลง", command=convert)
convert_button.grid(row=0, column=2, padx=10, pady=10)

# ส่วนแสดงผล
result_label = tk.Label(root, text="ผลลัพธ์: ", font=("Arial", 12))
result_label.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

# เริ่มต้นโปรแกรม
root.mainloop()
